Data Capture with RealSense

In [ ]:
import open3d as o3d
import pyrealsense2 as rs
import numpy as np
from enum import IntEnum
import open3d as o3d
import keyboard
import json
import cv2
def rbgdmaker(s):
    depth = o3d.io.read_image("depth"+str(s)+".png")
    color = o3d.io.read_image("color"+str(s)+".jpg")
    rgbd=o3d.geometry.RGBDImage.create_from_color_and_depth(color, depth, depth_scale=1000, depth_trunc=1.0, convert_rgb_to_intensity=False)
    intrinsic=o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd,intrinsic)
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    o3d.visualization.draw_geometries([pcd])
class Preset(IntEnum):
    Custom = 0
    Default = 1
    Hand = 2
    HighAccuracy = 3
    HighDensity = 4
    MediumDensity = 5
def save_intrinsic_as_json(filename, frame):
    intrinsics = frame.profile.as_video_stream_profile().intrinsics
    with open(filename, 'w') as outfile:
        obj = json.dump(
            {
                'width':
                    intrinsics.width,
                'height':
                    intrinsics.height,
                'intrinsic_matrix': [
                    intrinsics.fx, 0, 0, 0, intrinsics.fy, 0, intrinsics.ppx,
                    intrinsics.ppy, 1
                ]
            },
            outfile,
            indent=4)
        
pipeline = rs.pipeline()
config = rs.config()
profile = pipeline.start(config)
depth_sensor = profile.get_device().first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, Preset.HighAccuracy)
depth_scale = depth_sensor.get_depth_scale()
clipping_distance_in_meters = 1.0
clipping_distance = clipping_distance_in_meters / depth_scale
print("depth_scale=",depth_scale)
align_to = rs.stream.color
align = rs.align(align_to)
frame_count = 0
while True:
    frames = pipeline.wait_for_frames()
    aligned_frames = align.process(frames)
    aligned_depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()
    if not aligned_depth_frame or not color_frame:
                continue
    depth_image = np.asanyarray(aligned_depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())
    if frame_count == 0:
                    save_intrinsic_as_json(
                        "camera_intrinsic.json",
                        color_frame)
    cv2.imwrite("depth"+str(frame_count)+".png" , depth_image)
    cv2.imwrite("color"+str(frame_count)+".jpg" , color_image)
    print("Saved color + depth image %06d" % frame_count)
    
    grey_color = 153
    depth_image_3d = np.dstack((depth_image, depth_image, depth_image))
    bg_removed = np.where((depth_image_3d > clipping_distance) | \
                    (depth_image_3d <= 0), grey_color, color_image)

    depth_colormap = cv2.applyColorMap(
                cv2.convertScaleAbs(depth_image, alpha=0.09), cv2.COLORMAP_JET)
    images = np.hstack((bg_removed, depth_colormap))
    cv2.namedWindow('Recorder Realsense', cv2.WINDOW_AUTOSIZE)
    cv2.imshow('Recorder Realsense', images)
    cv2.waitKey(1)
    rbgdmaker(frame_count)
    frame_count += 1
    if keyboard == "q":
                cv2.destroyAllWindows()
                break
    # frame_count += 1
pipeline.stop()


Reconstruction Part

In [3]:
import math
import numpy as np
import open3d as o3d

n_files = 38 #----------------Number of Color Files-----------------------------------
n_frames_per_fragment=2 #---------------------Keep this value minimum for more flexibilty

def rbgdmaker(s):
    depth = o3d.io.read_image("depth"+str(s)+".png")
    color = o3d.io.read_image("color"+str(s)+".jpg")
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(color, depth,depth_scale=1000, depth_trunc=1.0,convert_rgb_to_intensity=False)
    return rgbd

def register_one_rgbd_pair(s, t, intrinsic):
    source_rgbd_image = rbgdmaker(s)
    print("Source obtained",s)
    target_rgbd_image = rbgdmaker(t)        
    print("Source obtained",t)
    
    option = o3d.pipelines.odometry.OdometryOption()
    option.depth_diff_max = 0.07
    odo_init = np.identity(4)
    [success, trans, info] = o3d.pipelines.odometry.compute_rgbd_odometry(
            source_rgbd_image, target_rgbd_image, intrinsic, odo_init,
            o3d.pipelines.odometry.RGBDOdometryJacobianFromHybridTerm(), option)
    return [success, trans, info]
posegraphsss=[]
def make_posegraph_for_fragment( sid, eid, fragment_id, n_fragments,
                                intrinsic):
        o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)
        pose_graph =o3d.pipelines.registration.PoseGraph()
        trans_odometry = np.identity(4)
        pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(trans_odometry))
        o=0
        for s in range(sid, eid):
                for t in range(s + 1, eid): #####changed s+1 to s+2
                        if t == s + 1: ####same
                                print(
                    "Fragment %03d / %03d :: RGBD matching between frame : %d and %d"
                    % (fragment_id, n_fragments - 1, s, t))
                                [success, trans,
                 info] = register_one_rgbd_pair(s, t, 
                                                intrinsic)
                                trans_odometry = np.dot(trans, trans_odometry)
                                
                                
                                trans_odometry_inv = np.linalg.inv(trans_odometry)
                                pose_graph.nodes.append(
                    o3d.pipelines.registration.PoseGraphNode(
                        trans_odometry_inv))
                                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(s - sid,
                                                             t - sid,
                                                             trans,
                                                             info,
                                                             uncertain=False))
        print("Pose graph stage")
        posegraphsss.append(pose_graph)
        o3d.io.write_pose_graph(str(fragment_id)+"posegraph.json",pose_graph)
        return pose_graph                         

def integrate_rgb_frames_for_fragment( pose_graph,fragment_id,n_fragments, intrinsic):
    volume = o3d.pipelines.integration.ScalableTSDFVolume(
        voxel_length=3.0 / 512.0,
        sdf_trunc=0.04,
        color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)
    for i in range(len(pose_graph.nodes)):
        i_abs = fragment_id * n_frames_per_fragment + i
        rgbd = rbgdmaker(i_abs)
        pose = pose_graph.nodes[i].pose
        volume.integrate(rgbd, intrinsic, np.linalg.inv(pose))
    mesh = volume.extract_triangle_mesh()
    mesh.compute_vertex_normals()
    return mesh

def make_pointcloud_for_fragment(pose_graph,fragment_id, n_fragments, intrinsic):
    mesh = integrate_rgb_frames_for_fragment(pose_graph,fragment_id,n_fragments,intrinsic)
    pcd = o3d.geometry.PointCloud()
    pcd.points = mesh.vertices
    pcd.colors = mesh.vertex_colors
    pcd_name = str(fragment_id)+"pointcloud.ply"
    o3d.io.write_point_cloud(pcd_name, pcd, False, True)

def process_single_fragment(fragment_id, n_files,n_fragments, n_frames_per_fragment):
            intrinsic = o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)
            sid = fragment_id * n_frames_per_fragment
            print("Sid=",sid)
            eid = min(sid + n_frames_per_fragment, n_files)
            print("eid=",eid)
            pose_graph=make_posegraph_for_fragment( sid, eid, fragment_id, n_fragments,
                                    intrinsic)
            print("PointCLoud Stage")
            make_pointcloud_for_fragment( pose_graph,fragment_id, n_fragments,intrinsic)
            print("PointCLoud Done")



n_fragments = int(math.ceil(float(n_files) / n_frames_per_fragment))
print("n fragments=",n_fragments)
for fragment_id in range(n_fragments):
    process_single_fragment(fragment_id,n_files, n_fragments,n_frames_per_fragment )

import numpy as np
import open3d as o3d
import math
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
ply_file_names = []
for i in range(n_fragments-6):
    k=str(i)+"pointcloud.ply"
    ply_file_names.append(k)
print(ply_file_names)

class matching_result:
    def __init__(self, s, t):
        self.s = s
        self.t = t
        self.success = False
        self.transformation = np.identity(4)
        self.infomation = np.identity(6)
def update_posegraph_for_scene(s, t, transformation, information, odometry,
                               pose_graph):
    if t == s + 1:  # odometry case
        odometry = np.dot(transformation, odometry)
        odometry_inv = np.linalg.inv(odometry)
        pose_graph.nodes.append(
            o3d.pipelines.registration.PoseGraphNode(odometry_inv))
        pose_graph.edges.append(
            o3d.pipelines.registration.PoseGraphEdge(s,t,transformation,
                                                     information,uncertain=False))
    else:  # loop closure case
        pose_graph.edges.append(
            o3d.pipelines.registration.PoseGraphEdge(s,
                                                     t,
                                                     transformation,
                                                     information,
                                                     uncertain=True))
    return (odometry, pose_graph)

def multiscale_icp(source,target,voxel_size,max_iter,init_transformation=np.identity(4)):
    current_transformation = init_transformation
    for i, scale in enumerate(range(len(max_iter))):  #
        iter = max_iter[scale]
        voxel_size=0.05
        distance_threshold = voxel_size * 1.4
        source_down = source.voxel_down_sample(voxel_size)
        target_down = target.voxel_down_sample(voxel_size)
        result_icp = o3d.pipelines.registration.registration_generalized_icp(
                    source_down, target_down, distance_threshold,
                    current_transformation,
                    o3d.pipelines.registration.
                    TransformationEstimationForGeneralizedICP(),
                    o3d.pipelines.registration.ICPConvergenceCriteria(
                        relative_fitness=1e-6,
                        relative_rmse=1e-6,
                        max_iteration=iter))
        current_transformation = result_icp.transformation
        if i == len(max_iter) - 1:
            information_matrix = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
                source_down, target_down, voxel_size * 1.4,
                result_icp.transformation)
    return (result_icp.transformation, information_matrix)

voxel_size=0.05
def register_point_cloud_fpfh(source, target, source_fpfh, target_fpfh):
    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
    distance_threshold = voxel_size * 1.4

    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source, target, source_fpfh, target_fpfh, False, distance_threshold,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(
                False), 4,
            [
                o3d.pipelines.registration.
                CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                    distance_threshold)
            ],
            o3d.pipelines.registration.RANSACConvergenceCriteria(
                1000000, 0.999))
    if (result.transformation.trace() == 4.0):
        return (False, np.identity(4), np.zeros((6, 6)))
    information = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
        source, target, distance_threshold, result.transformation)
    if information[5, 5] / min(len(source.points), len(target.points)) < 0.3:
        return (False, np.identity(4), np.zeros((6, 6)))
    return (True, result.transformation, information)

def compute_initial_registration(s, t, source_down, target_down, source_fpfh,
                                 target_fpfh):

    if t == s + 1:  # odometry case
        print("Using RGBD odometry")
        pose_graph_frag = o3d.io.read_pose_graph(str(t)+"posegraph.json") #-----------------
        n_nodes = len(pose_graph_frag.nodes)
        transformation_init = np.linalg.inv(pose_graph_frag.nodes[n_nodes -
                                                                  1].pose)
        (transformation, information) = \
                multiscale_icp(source_down, target_down,
                [0.05], [50], transformation_init)
    else:  # loop closure case
        (success, transformation,
         information) = register_point_cloud_fpfh(source_down, target_down,
                                                  source_fpfh, target_fpfh,
                                                  )
        if not success:
            print("No reasonable solution. Skip this pair")
            return (False, np.identity(4), np.zeros((6, 6)))
    print(transformation)


   
    return (True, transformation, information)

def preprocess_point_cloud(pcd):
    voxel_size = 0.05
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0,
                                             max_nn=30))
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5.0,
                                             max_nn=100))
    return (pcd_down, pcd_fpfh)

def register_point_cloud_pair(ply_file_names, s, t):
    print("reading %s ..." % ply_file_names[s])
    source = o3d.io.read_point_cloud(ply_file_names[s])
    print("reading %s ..." % ply_file_names[t])
    target = o3d.io.read_point_cloud(ply_file_names[t])
    (source_down, source_fpfh) = preprocess_point_cloud(source)
    (target_down, target_fpfh) = preprocess_point_cloud(target)
    (success, transformation, information) =compute_initial_registration(
            s, t, source_down, target_down,
            source_fpfh, target_fpfh)
    return (success, transformation, information)
def make_posegraph_for_scene(ply_file_names):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_files = len(ply_file_names)
    matching_results = {}
    for s in range(n_files):
        for t in range(s + 1, n_files):
            matching_results[s * n_files + t] = matching_result(s, t)
    for r in matching_results:
        (matching_results[r].success, matching_results[r].transformation,
                    matching_results[r].information)=register_point_cloud_pair(ply_file_names,
                    matching_results[r].s, matching_results[r].t)
    for r in matching_results:
        if matching_results[r].success:
            (odometry, pose_graph) = update_posegraph_for_scene(
                matching_results[r].s, matching_results[r].t,
                matching_results[r].transformation,
                matching_results[r].information, odometry, pose_graph)
    return pose_graph

jpose=make_posegraph_for_scene(ply_file_names)

def rbgdmaker(s):
    depth = o3d.io.read_image("depth"+str(s)+".png")
    color = o3d.io.read_image("color"+str(s)+".jpg")
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(color, depth,depth_scale=1000, depth_trunc=1.0,convert_rgb_to_intensity=False)
    return rgbd
intrinsic = o3d.camera.PinholeCameraIntrinsic(
            o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)
def scalable_integrate_rgb_frames(pose_graph,intrinsic,n_files,n_frames_per_fragment):
    poses = []
    n_fragments = int(math.ceil(float(n_files) /n_frames_per_fragment))
    volume = o3d.pipelines.integration.ScalableTSDFVolume(
        voxel_length=3.0 / 512.0,
        sdf_trunc=0.04,
        color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)
    pose_graph_fragment = pose_graph
    for fragment_id in range(len(pose_graph_fragment.nodes)):
        pose_graph_rgbd = o3d.io.read_pose_graph(str(fragment_id)+"posegraph.json") #------------------------------------
        for frame_id in range(len(pose_graph_rgbd.nodes)):
            frame_id_abs = fragment_id *  n_frames_per_fragment + frame_id
            print(
                "Fragment %03d / %03d :: integrate rgbd frame %d (%d of %d)." %
                (fragment_id, n_fragments - 1, frame_id_abs, frame_id + 1,
                 len(pose_graph_rgbd.nodes)))
            rgbd = rbgdmaker(frame_id_abs)
            pose = np.dot(pose_graph_fragment.nodes[fragment_id].pose,
                          pose_graph_rgbd.nodes[frame_id].pose)
            volume.integrate(rgbd, intrinsic, np.linalg.inv(pose))
            poses.append(pose)
    mesh = volume.extract_triangle_mesh()
    mesh.compute_vertex_normals()
    mesh.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    mesh = mesh.subdivide_loop(number_of_iterations=3)
    o3d.io.write_triangle_mesh("Full.ply", mesh, False, True)
    o3d.visualization.draw_geometries([mesh])
scalable_integrate_rgb_frames(jpose,intrinsic,n_files,n_frames_per_fragment)

n fragments= 19
Sid= 0
eid= 2
Fragment 000 / 018 :: RGBD matching between frame : 0 and 1
Source obtained 0
Source obtained 1
Pose graph stage
PointCLoud Stage
PointCLoud Done
Sid= 2
eid= 4
Fragment 001 / 018 :: RGBD matching between frame : 2 and 3
Source obtained 2
Source obtained 3
Pose graph stage
PointCLoud Stage
PointCLoud Done
Sid= 4
eid= 6
Fragment 002 / 018 :: RGBD matching between frame : 4 and 5
Source obtained 4
Source obtained 5
Pose graph stage
PointCLoud Stage
PointCLoud Done
Sid= 6
eid= 8
Fragment 003 / 018 :: RGBD matching between frame : 6 and 7
Source obtained 6
Source obtained 7
Pose graph stage
PointCLoud Stage
PointCLoud Done
Sid= 8
eid= 10
Fragment 004 / 018 :: RGBD matching between frame : 8 and 9
Source obtained 8
Source obtained 9
Pose graph stage
PointCLoud Stage
PointCLoud Done
Sid= 10
eid= 12
Fragment 005 / 018 :: RGBD matching between frame : 10 and 11
Source obtained 10
Source obtained 11
Pose graph stage
PointCLoud Stage
PointCLoud Done
Sid= 12
eid= 14
F